In [1]:
%%time
import sagemaker
import boto3
from sagemaker.model_monitor import DataCaptureConfig, DefaultModelMonitor, DatasetFormat
from sagemaker.model import Model
from botocore.exceptions import ClientError


from datetime import datetime, timedelta, timezone
import json
import os
import re
import boto3
from time import sleep
from threading import Thread

import pandas as pd

from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer

from sagemaker.model import Model
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

session = Session()
region = boto3.Session().region_name

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
CPU times: user 1.21 s, sys: 245 ms, total: 1.45 s
Wall time: 1.55 s


In [2]:
import xgboost

In [3]:
%store -r model_data
%store -r bucket_name
%store -r role
%store -r pipeline_definition

train_data = f's3://{bucket_name}/processed/housing/train.csv'
validation_data = f's3://{bucket_name}/processed/housing/validation.csv'
test_data = f's3://{bucket_name}/processed/housing/test.csv'

In [4]:
prefix = "housing-ModelQualityMonitor-20240623"

##S3 prefixes
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"

ground_truth_upload_path = (
    f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

endpoint_name = f"housing-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

model_name = f"housing-pred-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

image_uri = image_uris.retrieve(framework="xgboost", version="0.90-1", region=region)

model = Model(image_uri=image_uri, model_data=model_data, role=role, sagemaker_session=session)

model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = housing-model-quality-monitor-2024-06-24-0326
------!

In [5]:
from sagemaker.predictor import Predictor

predictor = Predictor(
    endpoint_name=endpoint_name, sagemaker_session=session, serializer=CSVSerializer()
)

In [6]:
df = pd.read_csv(validation_data)
df.head()

,mssubclass,lotarea,yearbuilt,yearremodadd,masvnrarea,bsmtfinsf1,bsmtfinsf2,bsmtunfsf,totalbsmtsf,firstflrsf,...,openporchsf,enclosedporch,threessnporch,screenporch,poolarea,miscval,mosold,yrsold,saleprice,logsaleprice
0,50.0,7425.0,1945.0,1950.0,0.0,0.0,0.0,672.0,672.0,1195.0,...,0.0,0.0,0.0,0.0,0.0,0.0,8.0,2008.0,163000.0,12.001506
1,160.0,1504.0,1972.0,1972.0,0.0,252.0,0.0,294.0,546.0,546.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,2007.0,163000.0,12.001506
2,60.0,16259.0,2006.0,2006.0,370.0,0.0,0.0,1249.0,1249.0,1249.0,...,154.0,0.0,0.0,0.0,0.0,0.0,9.0,2006.0,342643.0,12.744445
3,20.0,11500.0,1957.0,1957.0,0.0,0.0,0.0,0.0,0.0,845.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2009.0,84000.0,11.338573
4,20.0,9600.0,1950.0,2006.0,0.0,32.0,308.0,232.0,572.0,1337.0,...,192.0,0.0,0.0,0.0,0.0,0.0,9.0,2008.0,163000.0,12.001506


In [7]:
import pandas as pd
import xgboost
from sagemaker.predictor import Predictor
from sagemaker.serializers import CSVSerializer

In [8]:
df = pd.read_csv(validation_data)
y_validation = df['saleprice'].to_numpy()
X_validation = df.drop(columns=['saleprice'])
#.values


# Prepare a list to store predictions
predictions = []

# Iterate over each row in the validation DataFrame
for _, row in X_validation.iterrows():
    # Convert the row to CSV format
    row_csv = row.to_frame().T.to_csv(index=False, header=False)
    # Get prediction from the model endpoint
    prediction = predictor.predict(row_csv).decode('utf-8')
    # Convert prediction to float and append to the list
    predictions.append(float(prediction))

# Add predictions to the validation dataset
df['predictions'] = predictions

#dvalidation = xgboost.DMatrix(X_validation)
#predictions = model.predict(dvalidation)

# Add predictions to the validation dataset
#df['predictions'] = predictions
# Save the modified validation dataset to a new CSV
local_modified_validation_data = "modified_validation.csv"
#df.to_csv(local_modified_validation_data, index=False)




ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/housing-model-quality-monitor-2024-06-24-0326 in account 952054755114 for more information.

In [9]:
df = pd.read_csv(validation_data)
y_validation = df['saleprice'].to_numpy()
X_validation = df.drop(columns=['saleprice'])
#.values


predictions = predictor.predict(X_validation.to_csv(index=False, header=False)).decode('utf-8')
predictions = [float(p) for p in predictions.split(',')]

# Add predictions to the validation dataset
df['predictions'] = predictions

#dvalidation = xgboost.DMatrix(X_validation)
#predictions = model.predict(dvalidation)

# Add predictions to the validation dataset
#df['predictions'] = predictions
# Save the modified validation dataset to a new CSV
local_modified_validation_data = "modified_validation.csv"
#df.to_csv(local_modified_validation_data, index=False)




ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (502) from primary with message "<html>
<head><title>502 Bad Gateway</title></head>
<body bgcolor="white">
<center><h1>502 Bad Gateway</h1></center>
<hr><center>nginx/1.10.3 (Ubuntu)</center>
</body>
</html>
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/housing-model-quality-monitor-2024-06-24-0326 in account 952054755114 for more information.

In [ ]:
df.to_csv(f"s3://{bucket_name}/processed/housing/modified_validation.csv", header=True,  index=False)

In [ ]:
# Define the S3 URI for the modified validation dataset
baseline_data_uri = f"s3://{bucket_name}/processed/housing/modified_validation.csv"

# Upload the modified validation dataset to S3
baseline_dataset_uri = S3Uploader.upload(local_modified_validation_data, baseline_data_uri)

In [32]:
baseline_prefix = prefix + "/baselining"
baseline_results_prefix = baseline_prefix + "/results"

baseline_dataset_uri = modified_validation_data_path
baseline_results_uri = f"s3://{bucket_name}/{baseline_results_prefix}"

In [33]:
# Create the model quality monitoring object
housing_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [34]:
# Name of the model quality baseline job
baseline_job_name = f"housing-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"

In [35]:
# Execute the baseline suggestion job.
job = housing_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="Regression",
    inference_attribute="prediction",
    ground_truth_attribute="saleprice",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name housing-model-baseline-job-2024-06-24-0148


...........................................................*

UnexpectedStatusException: Error for Processing job housing-model-baseline-job-2024-06-24-0148: Failed. Reason: AlgorithmError: Error: Errors occurred when analyzing your data. Please check CloudWatch logs for more details., exit code: 255

In [ ]:
baseline_job = housing_model_quality_monitor.latest_baselining_job


In [ ]:
binary_metrics = baseline_job.baseline_statistics().body_dict["binary_classification_metrics"]
pd.json_normalize(binary_metrics).T
pd.DataFrame(baseline_job.suggested_constraints().body_dict["binary_classification_constraints"]).T


In [ ]:
def invoke_endpoint(ep_name, file_name):
    with open(file_name, "r") as f:
        i = 0
        for row in f:
            payload = row.rstrip("\n")
            response = session.sagemaker_runtime_client.invoke_endpoint(
                EndpointName=endpoint_name,
                ContentType="text/csv",
                Body=payload,
                InferenceId=str(i),  # unique ID per row
            )["Body"].read()
            i += 1
            sleep(1)


def invoke_endpoint_forever():
    while True:
        try:
            invoke_endpoint(endpoint_name, "test_data/test-dataset-input-cols.csv")
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever)
thread.start()

In [10]:
session = sagemaker.Session()
region = boto3.Session().region_name

endpoint_name = 'housing-endpoint'

xgboost_image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    image_scope="training"
)

sm_client = boto3.client('sagemaker')

# Create a Sagemaker Model and deplot to endpoint if it does not exist
try:
    response = sm_client.describe_endpoint(EndpointName=endpoint_name)
    endpoint_exists = response['EndpointStatus'] in ['InService', 'Creating', 'Updating']
except ClientError as e:
    if e.response['Error']['Code'] == 'ValidationException':
        endpoint_exists = False
    else:
        raise e

if not endpoint_exists:
    print(f"Endpoint '{endpoint_name}' does not exist. Deploying the model...")
    model = Model(model_data=model_data, role=role, image_uri=xgboost_image_uri)
    predictor = model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name,
        wait=True,
    )
    model_name = model.name
    print(f"Model deployed to endpoint: {endpoint_name}")
else:
    print(f"Endpoint '{endpoint_name}' already exists. Skipping deployment...")
    model = Model(model_data=model_data, role=role, image_uri=xgboost_image_uri)
    model_name = model.name
    predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker.Session())


Endpoint 'housing-endpoint' already exists. Skipping deployment...


In [11]:
# Define the S3 location for captured data
data_capture_s3_uri = f's3://{bucket_name}/data-capture/'
print(f"Data capture S3 URI: {data_capture_s3_uri}")



# Create a DataCaptureConfig object
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=data_capture_s3_uri,
    capture_options=["REQUEST", "RESPONSE"]
)
print("DataCaptureConfig created.")

Data capture S3 URI: s3://housing-dataset-5435v3/data-capture/
DataCaptureConfig created.


In [12]:
# Create a new endpoint configuration with data capture enabled

endpoint_config_name = f"{endpoint_name}-config"
endpoint_config_response = session.sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[{
        'VariantName': 'AllTraffic',
        'ModelName': model_name,
        'InitialInstanceCount': 1,
        'InstanceType': 'ml.m5.xlarge',
        'InitialVariantWeight': 1
    }],
    DataCaptureConfig=data_capture_config._to_request_dict()
)
print("Endpoint configuration created.")

# Update the endpoint with the new configuration
sm_client.update_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print(f"Endpoint '{endpoint_name}' updated with data capture configuration.")


ParamValidationError: Parameter validation failed:
Invalid type for parameter ProductionVariants[0].ModelName, value: None, type: <class 'NoneType'>, valid types: <class 'str'>

In [42]:
# Define the S3 location for the baseline data
baseline_data_uri = train_data
print(f"Baseline data S3 URI: {baseline_data_uri}")

Baseline data S3 URI: s3://housing-dataset-5435v3/processed/housing/train.csv


In [35]:
# Create a DefaultModelMonitor object
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=100,
    max_runtime_in_seconds=3600,
    sagemaker_session=session
)
print("DefaultModelMonitor created.")

DefaultModelMonitor created.


In [38]:
# Create a baseline from the training data
monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_data_uri
)
print("Baseline suggested from training data.")


INFO:sagemaker:Creating processing-job with name baseline-suggestion-job-2024-06-23-22-43-56-980


ClientError: An error occurred (ValidationException) when calling the CreateProcessingJob operation: No S3 objects found under S3 URL "s3://housing-dataset-5435v3/baseline-data/" given in input data source. Please ensure that the bucket exists in the selected region (us-east-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::952054755114:role/LabRole" has "s3:ListBucket" permissions on bucket "housing-dataset-5435v3".

In [32]:



# Create a DefaultModelMonitor object
monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
    volume_size_in_gb=100,
    max_runtime_in_seconds=3600,
    sagemaker_session=session
)

# Create a baseline from the training data
monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_data_uri
)

# Create a monitoring schedule
monitor.create_monitoring_schedule(
    endpoint_input=predictor.endpoint_name,
    output_s3_uri=f's3://{bucket_name}/monitoring-output/',
    schedule_cron_expression="cron(0 * * * ? *)",  # Run every hour
    statistics=monitor.baseline_statistics(),
    constraints=monitor.suggested_constraints()
)

# List the monitoring reports
s3_client = boto3.client('s3')
response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix='monitoring-output/')

# Download and analyze the reports
for obj in response.get('Contents', []):
    report_key = obj['Key']
    s3_client.download_file(bucket_name, report_key, report_key.split('/')[-1])
    # Add your analysis code here

print("Model monitoring setup complete.")

NameError: name 'DatasetFormat' is not defined